# Description

- Initialize with returns, predictions, target volatility, and oos start date
- Evaluate portfolios generated from the predictions

- TODO(gp): This should be called `Master_model_evaluator` like the class

# Imports

In [2]:
%load_ext autoreload
%autoreload 2

import logging

import core.config as cconfig
import dataflow.model.model_evaluator as dtfmomoeva
import dataflow.model.model_plotter as dtfmomoplo
import helpers.dbg as hdbg
import helpers.printing as hprint

In [3]:
hdbg.init_logger(verbosity=logging.INFO)
# hdbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

# _LOG.info("%s", env.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-bbac118b-4779-4694-ba22-9de6f26b2b89.json'


# Notebook config

In [6]:
# Read from env var.
eval_config = cconfig.Config.from_env_var("AM_CONFIG_CODE")

# Override config.
if eval_config is None:
    experiment_dir = "/cache/experiments/oos_experiment.RH1E.v2_0-top100.5T"
    # experiment_dir = "/app/rc_experiment.RH8Ec.v2_0-top2.5T.2009.run1"
    aws_profile = None
    selected_idxs = None

    eval_config = cconfig.get_config_from_nested_dict(
        {
            "load_experiment_kwargs": {
                "src_dir": experiment_dir,
                "file_name": "result_bundle.v2_0.pkl",
                "experiment_type": "ins_oos",
                "selected_idxs": selected_idxs,
                "aws_profile": aws_profile,
            },
            "model_evaluator_kwargs": {
                "predictions_col": "mid_ret_0_vol_adj_clipped_2_hat",
                "target_col": "mid_ret_0_vol_adj_clipped_2",
                # "oos_start": "2017-01-01",
                "oos_start": None,
                "abort_on_error": True,
            },
            "bh_adj_threshold": 0.1,
            "resample_rule": "W",
            "mode": "ins",
            "target_volatility": 0.1,
        }
    )

print(str(eval_config))

Environment variable 'AM_CONFIG_CODE' not defined: no config retrieved
load_experiment_kwargs:
  src_dir: /cache/experiments/oos_experiment.RH1E.v2_0-top100.5T
  file_name: result_bundle.v2_0.pkl
  experiment_type: ins_oos
  selected_idxs: None
  aws_profile: None
model_evaluator_kwargs:
  predictions_col: mid_ret_0_vol_adj_clipped_2_hat
  target_col: mid_ret_0_vol_adj_clipped_2
  oos_start: None
  abort_on_error: True
bh_adj_threshold: 0.1
resample_rule: W
mode: ins
target_volatility: 0.1


# Initialize ModelEvaluator and ModelPlotter

In [7]:
# Build the ModelEvaluator from the eval config.
evaluator = dtfmomoeva.ModelEvaluator.from_eval_config(eval_config)

# Build the ModelPlotter.
plotter = dtfmomoplo.ModelPlotter(evaluator)

Before load_experiment_artifacts: memory_usage=rss=0.246GB vms=2.701GB mem_pct=0%
# Load artifacts 'result_bundle.v2_0.pkl' from '/cache/experiments/oos_experiment.RH1E.v2_0-top100.5T'
Found 100 experiment subdirs in '/cache/experiments/oos_experiment.RH1E.v2_0-top100.5T'


Loading artifacts:   0%|          | 0/100 [00:00<?, ?it/s]

load_experiment_artifacts: memory_usage=rss=0.332GB vms=3.419GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.376GB vms=3.463GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.458GB vms=3.604GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.524GB vms=3.697GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.540GB vms=3.725GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.563GB vms=3.772GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.615GB vms=3.873GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.631GB vms=3.897GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.604GB vms=3.897GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.619GB vms=3.925GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.641GB vms=3.995GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.681GB vms=4.057GB mem_pct=1%
load_experiment_artifacts: memory_usage=rss=0.661GB vms=4.057GB mem_pct=1%
load_experiment_artifacts

KeyboardInterrupt: 

# Analysis

In [ ]:
pnl_stats = evaluator.calculate_stats(
    mode=eval_config["mode"], target_volatility=eval_config["target_volatility"]
)
display(pnl_stats)

## Model selection

In [ ]:
plotter.plot_multiple_tests_adjustment(
    threshold=eval_config["bh_adj_threshold"], mode=eval_config["mode"]
)

In [ ]:
# TODO(gp): Move this chunk of code in a function.
col_mask = (
    pnl_stats.loc["signal_quality"].loc["sr.adj_pval"]
    < eval_config["bh_adj_threshold"]
)
selected = pnl_stats.loc[:, col_mask].columns.to_list()
not_selected = pnl_stats.loc[:, ~col_mask].columns.to_list()

print("num model selected=%s" % hprint.perc(len(selected), pnl_stats.shape[1]))
print("model selected=%s" % selected)
print("model not selected=%s" % not_selected)

# Use `selected = None` to show all the models.

In [ ]:
# selected = None
plotter.plot_multiple_pnls(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

## Return correlation

In [ ]:
plotter.plot_correlation_matrix(
    series="returns",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

In [ ]:
plotter.plot_effective_correlation_rank(
    series="returns",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

## Model correlation

In [ ]:
plotter.plot_correlation_matrix(
    series="pnl",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

In [ ]:
plotter.plot_effective_correlation_rank(
    series="pnl",
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
)

## Aggregate model

In [ ]:
pnl_srs, pos_srs, aggregate_stats = evaluator.aggregate_models(
    keys=selected,
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)
display(aggregate_stats)

In [ ]:
plotter.plot_sharpe_ratio_panel(keys=selected, mode=eval_config["mode"])

In [ ]:
plotter.plot_rets_signal_analysis(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
plotter.plot_performance(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
plotter.plot_rets_and_vol(
    keys=selected,
    resample_rule=eval_config["resample_rule"],
    mode=eval_config["mode"],
    target_volatility=eval_config["target_volatility"],
)

In [ ]:
if False:
    plotter.plot_positions(
        keys=selected,
        mode=eval_config["mode"],
        target_volatility=eval_config["target_volatility"],
    )

In [ ]:
if False:
    # Plot the returns and prediction for one or more models.
    model_key = selected[:1]
    plotter.plot_returns_and_predictions(
        keys=model_key,
        resample_rule=eval_config["resample_rule"],
        mode=eval_config["mode"],
    )